# Notebook to parallelize frame extraction

In [2]:
import os
import sys
import functools
from multiprocessing import Pool
import ffmpeg


In [8]:
def videos_to_frames(video_root, frame_root, num_threads=100):
    """videos_to_frames."""

    videos = []
    for r, d, f in os.walk(video_root):
        for file in f:
            if '.mp4' in file:
                videos.append(os.path.join(r, file))
#     print(videos)
    
    func = functools.partial(extract_frames, video_root=video_root, frame_root=frame_root)
    pool = Pool(num_threads)
    pool.map(func, videos)
    
    
def extract_frames(video, video_root='', frame_root='', tmpl='%05d.jpg', fps=30):
    name = os.path.basename(video)
    in_filename = video
    out_filename = os.path.join(frame_root, name, tmpl)
    
#     print(name,'\n',in_filename,'\n',out_filename)
    os.makedirs(os.path.dirname(out_filename), exist_ok=True)
    (
        ffmpeg
        .input(in_filename)
        .filter('fps', fps=fps, round='up')
        .output(out_filename)
        .run()
    )

In [9]:
video_root = 'fb_dfd_release_0.1_final/method_A'
frame_root = 'fb_dfd_release_0.1_final/method_A_frames2'

videos_to_frames(video_root, frame_root)